In [3]:
import os
import SimpleITK as sitk
from glob import glob


In [2]:
dataset_path = 'D:/DS18/data/BrainTumour'
modalities = ['T1', 'T1Gd', 'T2', 'FLAIR']

def Split4DNIFTY(inDir:str, outDir:str):

    for modality in modalities:
        os.makedirs(os.path.join(outDir, modality), exist_ok=True) #True not raise exception if exists

    #The glob function is used to find all the file paths matching a specified pattern
    nifti_files = glob(os.path.join(inDir, '4D', '*.nii.gz'))

    #print(nifti_files)

    #process each file
    for file_path in nifti_files:
        img_4d = sitk.ReadImage(file_path)
        size = list(img_4d.GetSize()) # Expected shape: [X, Y, Z, 4]
        #extract base name
        base_name = os.path.basename(file_path).replace('.nii.gz', '')

        for i, modality in enumerate(modalities):
            extract_size = list(img_4d.GetSize())[:3] + [0]
            extract_index = [0, 0, 0, i]
            img_3d = sitk.Extract(img_4d, extract_size, extract_index)
            output_path = os.path.join(outDir, modality, f'{base_name}_{modality}.nii.gz')
            sitk.WriteImage(img_3d, output_path)
            print(f'Saved: {output_path}')



In [3]:
# Split4DNIFTY(f'{dataset_path}/imagesTs', f'{dataset_path}/imagesTs')

In [ ]:


# Set directories for T1 and T2
t1_dir = r'D:/DS18/data/IXI-T1'
t2_dir = r'D:/DS18/data/IXI-T2'


t1_files = sorted(glob(os.path.join(t1_dir, "*.nii.gz")))
t2_files = sorted(glob(os.path.join(t2_dir, "*.nii.gz")))

# Ensure equal count
if len(t1_files) != len(t2_files):
    print(f"⚠️ Mismatch in number of files: {len(t1_files)} T1 vs {len(t2_files)} T2")

# Compare shapes
print("🔍 Checking for mismatched volume shapes...\n")
for t1_path, t2_path in zip(t1_files, t2_files):
    t1_shape = sitk.GetArrayFromImage(sitk.ReadImage(t1_path)).shape
    t2_shape = sitk.GetArrayFromImage(sitk.ReadImage(t2_path)).shape

    if t1_shape != t2_shape:
        print(f"❌ Shape mismatch:")
        print(f"  T1: {os.path.basename(t1_path)} -> {t1_shape}")
        print(f"  T2: {os.path.basename(t2_path)} -> {t2_shape}\n")


In [7]:
#resample 

# Input directories
input_dirs = {
    "Tumor_Labels": r'D:\DS18\data\BrainTumour\labelsTr'
    #"Tumor_T1": r'D:\DS18\data\BrainTumour\imagesTr\T1'
    #"IXI-T1": r'D:/DS18/data/IXI-T1',
    #"IXI-T2": r'D:/DS18/data/IXI-T2'
}

# Output directories
output_dirs = {
    "Tumor_Labels": r'D:\DS18\data\BrainTumour\labelsTr'
    #"Tumor_T1": r'D:\DS18\data\BrainTumour\imagesTr\T1_resampled'
    #"IXI-T1": r'D:/DS18/data/IXI-T1_resampled',
    #"IXI-T2": r'D:/DS18/data/IXI-T2_resampled'
}

# Target shape
target_size = (128, 256, 256)  # z, y, x

# Create output directories
for out_dir in output_dirs.values():
    os.makedirs(out_dir, exist_ok=True)

def resample_image(image, new_size):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    
    new_spacing = [
        original_spacing[i] * (original_size[i] / new_size[i])
        for i in range(3)
    ]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(new_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetInterpolator(sitk.sitkLinear)
    
    return resampler.Execute(image)

# Resample all images in both folders
for modality, input_dir in input_dirs.items():
    for file_path in glob(os.path.join(input_dir, "*.nii.gz")):
        img = sitk.ReadImage(file_path)
        resampled_img = resample_image(img, target_size)
        
        filename = os.path.basename(file_path)
        output_path = os.path.join(output_dirs[modality], filename)
        sitk.WriteImage(resampled_img, output_path)
        print(f"✅ Resampled {filename} to {target_size}")




✅ Resampled BRATS_001.nii.gz to (128, 256, 256)
✅ Resampled BRATS_002.nii.gz to (128, 256, 256)
✅ Resampled BRATS_003.nii.gz to (128, 256, 256)
✅ Resampled BRATS_004.nii.gz to (128, 256, 256)
✅ Resampled BRATS_005.nii.gz to (128, 256, 256)
✅ Resampled BRATS_006.nii.gz to (128, 256, 256)
✅ Resampled BRATS_007.nii.gz to (128, 256, 256)
✅ Resampled BRATS_008.nii.gz to (128, 256, 256)
✅ Resampled BRATS_009.nii.gz to (128, 256, 256)
✅ Resampled BRATS_010.nii.gz to (128, 256, 256)
✅ Resampled BRATS_011.nii.gz to (128, 256, 256)
✅ Resampled BRATS_012.nii.gz to (128, 256, 256)
✅ Resampled BRATS_013.nii.gz to (128, 256, 256)
✅ Resampled BRATS_014.nii.gz to (128, 256, 256)
✅ Resampled BRATS_015.nii.gz to (128, 256, 256)
✅ Resampled BRATS_016.nii.gz to (128, 256, 256)
✅ Resampled BRATS_017.nii.gz to (128, 256, 256)
✅ Resampled BRATS_018.nii.gz to (128, 256, 256)
✅ Resampled BRATS_019.nii.gz to (128, 256, 256)
✅ Resampled BRATS_020.nii.gz to (128, 256, 256)
✅ Resampled BRATS_021.nii.gz to (128, 25